In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import os

財報、月報酬，新舊合併csv轉feather

In [ ]:
folder_path = r'C:\Users\jason\單季'
file_path = os.path.join(folder_path, '單季.xlsx')
CSV = pd.read_excel(file_path)
CSV[['代號', '名稱']] = CSV['證券代碼'].str.split(' ', n=1, expand=True)
CSV.rename(columns={'年月':'年/月'}, inplace=True)
CSV.columns = CSV.columns.astype(str)
CSV['年/月'] = pd.to_datetime(CSV['年/月'].astype(str) + '01') + pd.offsets.MonthEnd(0)
CSV=CSV.drop(columns=['證券代碼'])

上下市公司處理

In [ ]:
folder_path = r"C:\Users\jason\公司上下市"
file_name = "上下市公司.csv"
file_name = os.path.join(folder_path,file_name)
df= pd.read_csv(file_name)
df['下市日期'] = df['下市日期'].astype(str).str.split('.').str[0]
df['下市日期'] = pd.to_datetime(df['下市日期'], format='%Y%m%d', errors='coerce')
df['最近上市日'] = pd.to_datetime(df['最近上市日'].astype(str), format='%Y%m%d')
df['最近上市日'] = df['最近上市日'] + pd.offsets.MonthEnd(0)
df.rename(columns={'簡稱':'名稱', '證券代碼':'代號'}, inplace=True)
df1 = df.pivot(index='最近上市日', columns='名稱', values='目前狀態')
df1

In [ ]:
df1= df1.reindex(pd.date_range(start='1962-02-28', end='2024-12-31', freq='M'))
### 根據 "最近上市日" 最早的時間，更新 index 的長度
### 資料區間為 1962(最早)~2024(最新)
### 消除儲存格中具有 current 的 value
df1[df1.notnull()] = np.NaN

### 查看
df1.notnull().sum()

In [ ]:
df2 = df1.stack().reset_index()
df2.columns = ['年/月', '名稱', '是否上市']
df_sort=df2.set_index(['名稱', '年/月'])[['是否上市']].dropna() \
    .sort_index(level=['名稱','年/月']).reset_index()
df_company=pd.merge(df_sort,df,on=['名稱'],how='inner').set_index(['代號', '名稱', '年/月'])[['是否上市']]
df_company=df_company.sort_index(level=['代號','年/月'])

大盤整理

In [ ]:
folder_path = r'C:\Users\jason\大盤每月'
file_path = os.path.join(folder_path, '20250308090942.csv')
y9999 = pd.read_csv(file_path)
y9999.rename(columns={'年月':'年/月'}, inplace=True)
y9999.columns = y9999.columns.astype(str)
y9999['年/月'] = pd.to_datetime(y9999['年/月'].astype(str) + '01', format='%Y%m%d')
y9999['年/月'] = y9999['年/月'] + pd.offsets.MonthEnd(0)
y9999=y9999.set_index(['證券代碼','年/月'])[['收盤價(元)_月','報酬率％_月']].reset_index()
